Copyright 2023 Borislav Rumenov Varbanov

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [4]:
import requests
import json
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

debug_bool = False
send_notification_bool = True

# Load configuration from JSON file
def load_config():
    with open('config.json', 'r') as f:
        return json.load(f)

In [5]:
# Send email notification
def send_notification(message, config):
    # Extract email settings from config
    from_email = config['FROM_EMAIL']
    to_email = config['TO_EMAIL']
    subject = "[Octopus] Daily Energy Prices Notification"

    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))

    # Gmail SMTP server settings
    smtp_server = config['SMTP_SERVER']
    smtp_port = config['SMTP_PORT']  # For TLS
    smtp_username = from_email
    smtp_password = config['SMTP_PASSWORD']

    # Establish a secure session
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)

    # Send email
    server.sendmail(from_email, to_email, msg.as_string())

    server.quit()


In [6]:
# Fetch and compare tariff prices
def fetch_and_compare_prices(config, TARIFF_CODE):
    # Initialize date-related variables
    today = datetime.now().date()
    date_str = today.strftime('%Y-%m-%d')  # Format date as YYYYMMDD
    historical_periods = {
        "yesterday": today - timedelta(days=1),
        "last_week": today - timedelta(weeks=1),
        "last_month": today - timedelta(days=30)
    }
    
    # Initialize price dictionaries
    current_prices = {}
    historical_prices = {key: {} for key in historical_periods.keys()}
    # Prepare email message
    message = f"Tarrif {TARIFF_CODE} price comparison for {date_str}:\n\n"
    # Fetch current day's data
    tomorrow_price_bool = True
    current_prices = fetch_prices(today, today, config, TARIFF_CODE)
    for label, prices in current_prices.items():
        message += f"Price {label}: {prices:.2f} p/kWh\n"
        today_price = prices
        if len(current_prices) > 1 and tomorrow_price_bool:
            tomorrow_price = prices
            tomorrow_price_bool = False
    
    # Fetch historical data and update message
    for label, date in historical_periods.items():
        historical_prices[label] = fetch_prices(date, today, config, TARIFF_CODE)

    # Compare and prepare notification message
    current_avg_price = sum(current_prices.values()) / len(current_prices)
    
    for label, prices in historical_prices.items():
        avg_price = sum(prices.values()) / len(prices)
        
        # Calculate percentage and absolute differences
        absolute_diff = today_price - avg_price
        percentage_diff = (absolute_diff / avg_price) * 100
        
        message += f"Average price {label}({historical_periods[label]}): {avg_price:.2f} p/kWh, Absolute Change: {absolute_diff:.2f} p/kWh, % Change: {percentage_diff:.2f}%\n"
    
    message += f"\nPrice today: {today_price:.2f} p/kWh\n"
    if tomorrow_price_bool is False:
        message += f"Price tomorrow: {tomorrow_price:.2f} p/kWh\n"

    if debug_bool:
        print("\nMessage debug print:\n", message)
    # Send notification
    if send_notification_bool:
        send_notification(message, config)

def fetch_prices(from_date, to_date, config, TARIFF_CODE):
    # API URL and parameters
    url = f"https://api.octopus.energy/v1/products/{config['PRODUCT_CODE']}/{TARIFF_CODE}/standard-unit-rates/"
    if debug_bool:
        print("\nURL debug pring:\n  ", url)
    params = {'period_from': f"{from_date}T00:00Z", 'period_to': f"{to_date}T23:59Z"}
    response = requests.get(url, params=params, auth=(config['API_KEY'], ""))
    
    # Handle API response
    if response.status_code != 200:
        print(f"Failed to fetch data for {from_date}. Status code: {response.status_code}")
        return {}
    
    data = response.json()
    prices = {}
    for rate in data['results']:
        time_period = f"{rate['valid_from']} to {rate['valid_to']}"
        prices[time_period] = rate['value_inc_vat']
        
    return prices

# Main execution
if __name__ == '__main__':
    try:
        config = load_config()
        if debug_bool:
            print("Config dict print:")
            for enum, value in config.items():
                print("  ", enum, " : ", value)
            print(f"\nProduct and tariff code debug print:\n  - Product code : ", config['PRODUCT_CODE'])
            for code, value in config['TARIFF_CODE'].items():
                print(f"  - {code} : {value}")
        for utility, code in config['TARIFF_CODE'].items():
            fetch_and_compare_prices(config, TARIFF_CODE=code)
    except Exception as e:
        print(f"An error occurred: {e}")
    # NOTE: Consider using bellow code if you'd like to run this as a cron job!
    # while True:
    #     try:
    #         ...
    #     except Exception as e:
    #         print(f"An error occurred: {e}")
    #     time.sleep(60*60*24)  # 24 hour